# UK House of Commons API 

- [https://developer.parliament.uk/](https://developer.parliament.uk/)


## Task 

Suppose we want to understand the types of questions that are being asked during Prime Minister's Questions (PMQs) in the UK House of Commons. We can use the UK House of Commons API to retrieve data about PMQs and analyze the types of questions being asked.

We will use the UK House of Commons API to retrieve data on PMQs. We will first learn how the API works, examining the output format and the available endpoints. Then, we will write a Python script to retrieve data on PMQs in 2025. Once we have the data, we will create a `pandas` DataFrame that contains a single row for each observation, with columns for the question text, the date it was asked, and the name of the MP who asked it. 

**Note:** Whenever we start collecting data from an API, we want to think about how we can use the data to answer a specific question. This often means that we should have in mind the type of analysis we want to perform, which will help us structure our data collection and analysis process.

In [2]:
pip install pandas requests

Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd 
import requests

# Step 1: Understand the API

- We need to identify the endpoints that provide data on PMQs. The url for the API documentation is [https://developer.parliament.uk/](https://developer.parliament.uk/).
- Once we have identified the endpoints, we can use tools like `curl` or `Postman` to test the API and understand the output format.
- The Endpoint for PMQs is `oralquestions`, which provides data on oral questions asked in the House of Commons.


The full URL for the PMQs endpoint is:
```
https://oralquestionsandmotions-api.parliament.uk/oralquestions/list
```



In [4]:
PMQs_endpoint = "https://oralquestionsandmotions-api.parliament.uk/oralquestions/list"

### We would usually like to write a function to get the data, but for simplicity, let's work through the collection process step by step


In [6]:
# send a GET request to the endpoint
r = requests.get(PMQs_endpoint)

# check if the request was successful
print(f"Status code: {r.status_code}")

Status code: 200


In [8]:
# let's convert the response to JSON
data = r.json()

In [14]:
# let's inspect the data
data

{'PagingInfo': {'Skip': 0,
  'Take': 40,
  'Total': 37601,
  'GlobalTotal': 37601,
  'StatusCounts': [],
  'GlobalStatusCounts': []},
 'StatusCode': 200,
 'Success': True,
 'Errors': [],
 'Response': [{'Id': 1207,
   'QuestionType': 1,
   'QuestionText': 'What recent steps she has taken to reduce the level of knife crime.',
   'Status': 5,
   'Number': 1,
   'TabledWhen': '2017-11-17T00:00:00',
   'RemovedFromToBeAskedWhen': None,
   'DeclarableInterestDetail': None,
   'HansardLink': '',
   'UIN': 901905,
   'AnsweringWhen': '2017-11-20T00:00:00',
   'AnsweringBodyId': 1,
   'AnsweringBody': 'Home Office',
   'AnsweringMinisterTitle': 'the Secretary of State for the Home Department',
   'AskingMember': {'MnisId': 4498,
    'PimsId': 6091,
    'Name': 'Mr Ranil Jayawardena',
    'ListAs': 'Jayawardena, Mr Ranil',
    'Constituency': 'North East Hampshire',
    'Status': 'Active',
    'Party': 'Conservative',
    'PartyId': 4,
    'PartyColour': None,
    'PhotoUrl': 'https://members-ap

In [ ]:
# we can call the 'PagingInfo' key to get information about the pagination
data['PagingInfo']

{'Skip': 0,
 'Take': 40,
 'Total': 37601,
 'GlobalTotal': 37601,
 'StatusCounts': [],
 'GlobalStatusCounts': []}

In [20]:
# to get the response data, we can access the 'Response' key
data['Response']

[{'Id': 1207,
  'QuestionType': 1,
  'QuestionText': 'What recent steps she has taken to reduce the level of knife crime.',
  'Status': 5,
  'Number': 1,
  'TabledWhen': '2017-11-17T00:00:00',
  'RemovedFromToBeAskedWhen': None,
  'DeclarableInterestDetail': None,
  'HansardLink': '',
  'UIN': 901905,
  'AnsweringWhen': '2017-11-20T00:00:00',
  'AnsweringBodyId': 1,
  'AnsweringBody': 'Home Office',
  'AnsweringMinisterTitle': 'the Secretary of State for the Home Department',
  'AskingMember': {'MnisId': 4498,
   'PimsId': 6091,
   'Name': 'Mr Ranil Jayawardena',
   'ListAs': 'Jayawardena, Mr Ranil',
   'Constituency': 'North East Hampshire',
   'Status': 'Active',
   'Party': 'Conservative',
   'PartyId': 4,
   'PartyColour': None,
   'PhotoUrl': 'https://members-api.parliament.uk/api/Members/4498/Portrait?cropType=OneOne&webVersion=true'},
  'AnsweringMinister': None,
  'AskingMemberId': 4498,
  'AnsweringMinisterId': 0},
 {'Id': 1232,
  'QuestionType': 2,
  'QuestionText': 'If she w

In [21]:
# how many responses are there?
len(data['Response'])

40

In [ ]:
# we can access each response in the list (python uses 0-based indexing)
data['Response'][0]

{'Id': 1207,
 'QuestionType': 1,
 'QuestionText': 'What recent steps she has taken to reduce the level of knife crime.',
 'Status': 5,
 'Number': 1,
 'TabledWhen': '2017-11-17T00:00:00',
 'RemovedFromToBeAskedWhen': None,
 'DeclarableInterestDetail': None,
 'HansardLink': '',
 'UIN': 901905,
 'AnsweringWhen': '2017-11-20T00:00:00',
 'AnsweringBodyId': 1,
 'AnsweringBody': 'Home Office',
 'AnsweringMinisterTitle': 'the Secretary of State for the Home Department',
 'AskingMember': {'MnisId': 4498,
  'PimsId': 6091,
  'Name': 'Mr Ranil Jayawardena',
  'ListAs': 'Jayawardena, Mr Ranil',
  'Constituency': 'North East Hampshire',
  'Status': 'Active',
  'Party': 'Conservative',
  'PartyId': 4,
  'PartyColour': None,
  'PhotoUrl': 'https://members-api.parliament.uk/api/Members/4498/Portrait?cropType=OneOne&webVersion=true'},
 'AnsweringMinister': None,
 'AskingMemberId': 4498,
 'AnsweringMinisterId': 0}

In [26]:
# let's write a function to convert the data into a pandas DataFrame
def pmqs_to_dataframe(data):
    # convert the list of responses to a DataFrame
    df = pd.DataFrame(data['Response'])
    
    return df

pmq_df = pmqs_to_dataframe(data)

In [27]:
pmq_df.head()

,Id,QuestionType,QuestionText,Status,Number,TabledWhen,RemovedFromToBeAskedWhen,DeclarableInterestDetail,HansardLink,UIN,AnsweringWhen,AnsweringBodyId,AnsweringBody,AnsweringMinisterTitle,AskingMember,AnsweringMinister,AskingMemberId,AnsweringMinisterId
0,1207,1,What recent steps she has taken to reduce the ...,5,1,2017-11-17T00:00:00,None,None,,901905,2017-11-20T00:00:00,1,Home Office,the Secretary of State for the Home Department,"{'MnisId': 4498, 'PimsId': 6091, 'Name': 'Mr R...",None,4498,0
1,1232,2,If she will make a statement on her department...,5,1,2017-11-17T00:00:00,None,None,,901895,2017-11-20T00:00:00,1,Home Office,the Secretary of State for the Home Department,"{'MnisId': 4664, 'PimsId': 6239, 'Name': 'Ms K...",None,4664,0
2,1208,1,What steps she is taking to reduce dangerous j...,5,2,2017-11-17T00:00:00,None,None,,901906,2017-11-20T00:00:00,1,Home Office,the Secretary of State for the Home Department,"{'MnisId': 4598, 'PimsId': 6219, 'Name': 'Moha...",None,4598,0
3,1233,2,If she will make a statement on her department...,7,2,2017-11-17T00:00:00,2017-11-17T12:47:47.527,None,,901896,2017-11-20T00:00:00,1,Home Office,the Secretary of State for the Home Department,"{'MnisId': 4453, 'PimsId': 6152, 'Name': 'Tomm...",None,4453,0
4,1209,1,What steps she is taking to reduce dangerous j...,5,3,2017-11-17T00:00:00,None,None,,901907,2017-11-20T00:00:00,1,Home Office,the Secretary of State for the Home Department,"{'MnisId': 481, 'PimsId': 1031, 'Name': 'Mr Ge...",None,481,0


## Step 2: Retrieve data on PMQs in 2025

Remember, we want to create a `pandas` DataFrame with all the PMQs in 2025. We'll need to use pagination to retrieve all the data, as the API may not return all results in a single request.

Oftentimes, good APIs will have features on the webpage that allow us to enter the parameters of our query, and then provide us with the URL that we can use to retrieve the data programmatically. We can use the API documentation to find out how to filter the data by date and other parameters. Doing so provides us with the following URL to retrieve PMQs for 2025:

```
https://oralquestionsandmotions-api.parliament.uk/oralquestions/list?parameters.answeringDateStart=2025-01-01
```

In [28]:
pmq_endpoint_2025 = 'https://oralquestionsandmotions-api.parliament.uk/oralquestions/list?parameters.answeringDateStart=2025-01-01'

In [29]:
r = requests.get(pmq_endpoint_2025)
# check if the request was successful
print(f"Status code: {r.status_code}")

Status code: 200


In [33]:
data_2025 = r.json()
pmq_df_2025 = pmqs_to_dataframe(data_2025)
pmq_df_2025.head()

,Id,QuestionType,QuestionText,Status,Number,TabledWhen,RemovedFromToBeAskedWhen,DeclarableInterestDetail,HansardLink,UIN,AnsweringWhen,AnsweringBodyId,AnsweringBody,AnsweringMinisterTitle,AskingMember,AnsweringMinister,AskingMemberId,AnsweringMinisterId
0,339423,2,If he will make a statement on his departmenta...,5,1,2024-12-19T00:00:00,None,None,,901993,2025-01-06T00:00:00,11,Ministry of Defence,the Secretary of State for Defence,"{'MnisId': 5301, 'PimsId': 6717, 'Name': 'Rebe...","{'MnisId': 400, 'PimsId': 909, 'Name': 'John H...",5301,400
1,339662,1,What steps he is taking to increase military s...,5,1,2024-12-19T00:00:00,None,None,,901968,2025-01-06T00:00:00,11,Ministry of Defence,the Secretary of State for Defence,"{'MnisId': 5289, 'PimsId': 6705, 'Name': 'Harp...","{'MnisId': 400, 'PimsId': 909, 'Name': 'John H...",5289,400
2,339661,2,If he will make a statement on his departmenta...,5,2,2024-12-19T00:00:00,None,None,,901994,2025-01-06T00:00:00,11,Ministry of Defence,the Secretary of State for Defence,"{'MnisId': 5289, 'PimsId': 6705, 'Name': 'Harp...","{'MnisId': 400, 'PimsId': 909, 'Name': 'John H...",5289,400
3,340355,1,What steps he is taking to ensure that SMEs ar...,5,2,2024-12-19T00:00:00,None,None,,901969,2025-01-06T00:00:00,11,Ministry of Defence,the Secretary of State for Defence,"{'MnisId': 4813, 'PimsId': 6353, 'Name': 'Mr R...","{'MnisId': 400, 'PimsId': 909, 'Name': 'John H...",4813,400
4,337796,2,If he will make a statement on his departmenta...,5,3,2024-12-19T00:00:00,None,None,,901995,2025-01-06T00:00:00,11,Ministry of Defence,the Secretary of State for Defence,"{'MnisId': 5161, 'PimsId': 6577, 'Name': 'Rich...","{'MnisId': 400, 'PimsId': 909, 'Name': 'John H...",5161,400


### APIs will often only return a limited number of results per request, so we need to handle pagination. The API documentation should tell us how to do this, but often it involves using a `page` parameter in the URL. We therefore need to loop through the pages until we have retrieved all the data.

In [35]:
# Here, we see that a single call will return a maximum of 40 results. We want to 'skip' the next 40 results to access the next page of results. This requires us to use a new parameter called 'skip'
data_2025['PagingInfo']

{'Skip': 0,
 'Take': 40,
 'Total': 2506,
 'GlobalTotal': 2506,
 'StatusCounts': [],
 'GlobalStatusCounts': []}

In [36]:
# we'll create a function to get the correct endpoint 

def get_pmq_endpoint(start_date, skip=0):
    """
    Function to get the PMQ endpoint for a given start date and skip value.
    
    Parameters:
    start_date (str): The start date in 'YYYY-MM-DD' format.
    skip (int): The number of results to skip.
    
    Returns:
    str: The PMQ endpoint URL.
    """
    return f'https://oralquestionsandmotions-api.parliament.uk/oralquestions/list?parameters.answeringDateStart={start_date}&parameters.skip={skip}'

# let's test the function
print(get_pmq_endpoint('2025-01-01', 0))  # First page
print(get_pmq_endpoint('2025-01-01', 40))  # Second page



https://oralquestionsandmotions-api.parliament.uk/oralquestions/list?parameters.answeringDateStart=2025-01-01&parameters.skip=0
https://oralquestionsandmotions-api.parliament.uk/oralquestions/list?parameters.answeringDateStart=2025-01-01&parameters.skip=40


In [ ]:
# now we'll create a function to loop through the pages to get all the results for a given start date

# it's 'polite' to not overload the server with requests, so we'll add a delay between requests
import time

def get_all_pmqs(start_date):
    """
    Function to get all PMQs from a given start date.
    
    Parameters:
    start_date (str): The start date in 'YYYY-MM-DD' format.
    
    Returns:
    pd.DataFrame: A DataFrame containing all PMQs.
    """
    all_pmqs = [] # initialize an empty list to store all PMQs
    skip = 0
    while True:
        endpoint = get_pmq_endpoint(start_date, skip)
        r = requests.get(endpoint)
        if r.status_code != 200:
            break
        data = r.json()
        if not data['Response']:
            break
        all_pmqs.extend(data['Response'])
        skip += 40  # Increment the skip value by 40 for the next page
        time.sleep(1) # to avoid overloading the server with requests
        print(f"Retrieved {len(data['Response'])} PMQs, total so far: {len(all_pmqs)}")
    return pd.DataFrame(all_pmqs)


# let's get all PMQs from 2025 ( I only used 2025-04-01 as an example, you can change it to any date you want)
pmqs_2025 = get_all_pmqs('2025-04-01')

Retrieved 40 PMQs, total so far: 40
Retrieved 40 PMQs, total so far: 80
Retrieved 40 PMQs, total so far: 120
Retrieved 40 PMQs, total so far: 160
Retrieved 40 PMQs, total so far: 200
Retrieved 40 PMQs, total so far: 240
Retrieved 40 PMQs, total so far: 280
Retrieved 40 PMQs, total so far: 320
Retrieved 40 PMQs, total so far: 360
Retrieved 40 PMQs, total so far: 400
Retrieved 40 PMQs, total so far: 440
Retrieved 40 PMQs, total so far: 480
Retrieved 40 PMQs, total so far: 520
Retrieved 40 PMQs, total so far: 560
Retrieved 40 PMQs, total so far: 600
Retrieved 40 PMQs, total so far: 640
Retrieved 40 PMQs, total so far: 680
Retrieved 40 PMQs, total so far: 720
Retrieved 40 PMQs, total so far: 760
Retrieved 40 PMQs, total so far: 800
Retrieved 40 PMQs, total so far: 840
Retrieved 40 PMQs, total so far: 880
Retrieved 40 PMQs, total so far: 920
Retrieved 15 PMQs, total so far: 935


In [45]:
# display the dataframe 
pmqs_2025

,Id,QuestionType,QuestionText,Status,Number,TabledWhen,RemovedFromToBeAskedWhen,DeclarableInterestDetail,HansardLink,UIN,AnsweringWhen,AnsweringBodyId,AnsweringBody,AnsweringMinisterTitle,AskingMember,AnsweringMinister,AskingMemberId,AnsweringMinisterId
0,359524,2,If he will make a statement on his departmenta...,5,1,2025-03-26T00:00:00,None,None,,903557,2025-04-01T00:00:00,208,"Foreign, Commonwealth and Development Office","the Secretary of State for Foreign, Commonweal...","{'MnisId': 4776, 'PimsId': 6418, 'Name': 'Muni...",None,4776,0
1,359585,1,What diplomatic steps he is taking to increase...,7,1,2025-03-26T00:00:00,2025-03-27T07:28:42.14,None,,903532,2025-04-01T00:00:00,208,"Foreign, Commonwealth and Development Office","the Secretary of State for Foreign, Commonweal...","{'MnisId': 4124, 'PimsId': 5617, 'Name': 'Chi ...",None,4124,0
2,358715,2,If he will make a statement on his departmenta...,5,2,2025-03-26T00:00:00,None,None,,903558,2025-04-01T00:00:00,208,"Foreign, Commonwealth and Development Office","the Secretary of State for Foreign, Commonweal...","{'MnisId': 345, 'PimsId': 1455, 'Name': 'Sir E...",None,345,0
3,359538,1,What assessment he has made of the potential i...,5,2,2025-03-26T00:00:00,None,None,,903533,2025-04-01T00:00:00,208,"Foreign, Commonwealth and Development Office","the Secretary of State for Foreign, Commonweal...","{'MnisId': 415, 'PimsId': 3727, 'Name': 'Fabia...",None,415,0
4,358019,1,Which development programmes he plans to maint...,8,3,2025-03-26T00:00:00,2025-03-27T08:01:00.67,None,,903534,2025-04-01T00:00:00,208,"Foreign, Commonwealth and Development Office","the Secretary of State for Foreign, Commonweal...","{'MnisId': 5358, 'PimsId': 6774, 'Name': 'John...",None,5358,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
930,367819,1,What recent progress he has made on establishi...,5,20,2025-05-22T00:00:00,None,None,,904377,2025-06-05T00:00:00,53,Cabinet Office,the Minister for the Cabinet Office,"{'MnisId': 4515, 'PimsId': 6132, 'Name': 'Kate...",None,4515,0
931,367484,1,What steps his Department is taking to reform ...,7,21,2025-05-22T00:00:00,2025-05-27T17:14:55.07,None,,904378,2025-06-05T00:00:00,53,Cabinet Office,the Minister for the Cabinet Office,"{'MnisId': 5081, 'PimsId': 6497, 'Name': 'Joe ...",None,5081,0
932,366391,1,What steps he has taken to encourage the insou...,5,22,2025-05-22T00:00:00,None,None,,904379,2025-06-05T00:00:00,53,Cabinet Office,the Minister for the Cabinet Office,"{'MnisId': 5232, 'PimsId': 6648, 'Name': 'Lorr...",None,5232,0
933,368011,1,What steps he is taking to improve relations w...,5,23,2025-05-22T00:00:00,None,None,,904380,2025-06-05T00:00:00,53,Cabinet Office,the Minister for the Cabinet Office,"{'MnisId': 5247, 'PimsId': 6663, 'Name': 'Soja...",None,5247,0


In [ ]:
# let's look at the columns in the DataFrame
pmqs_2025.columns

Index(['Id', 'QuestionType', 'QuestionText', 'Status', 'Number', 'TabledWhen',
       'RemovedFromToBeAskedWhen', 'DeclarableInterestDetail', 'HansardLink',
       'UIN', 'AnsweringWhen', 'AnsweringBodyId', 'AnsweringBody',
       'AnsweringMinisterTitle', 'AskingMember', 'AnsweringMinister',
       'AskingMemberId', 'AnsweringMinisterId'],
      dtype='object')

## Columns for the DataFrame

Remember, we want to create a `pandas` DataFrame with the following columns:

- `Date`: Date the question was asked
- `MP`: Name of the Member of Parliament (MP) who asked the question
- `Id`: Unique identifier for the question
- `QuestionText`: Text of the question
- `AnsweringBody`: The body that answered the question
- `AnsweringMinisterTitle`: Title of the minister who answered the question

Additionally, we can include any columns that are relevant to our analysis, such as the type of question or the topic.

One thing worth noting is that many APIs will return nested json objects. For example, the `AskingMember` field in the PMQs API returns an object with the MP's name, ID, constituency and other variables. We will need to extract the relevant information from these nested objects to create our DataFrame. See below for an example: 

In [51]:
pmqs_2025.AskingMember[0]

{'MnisId': 4776,
 'PimsId': 6418,
 'Name': 'Munira Wilson',
 'ListAs': 'Wilson, Munira',
 'Constituency': 'Twickenham',
 'Status': 'Active',
 'Party': 'Liberal Democrat',
 'PartyId': 17,
 'PartyColour': 'faa01a',
 'PhotoUrl': 'https://members-api.parliament.uk/api/Members/4776/Portrait?cropType=OneOne&webVersion=true'}

In [52]:
# convert the 'AskingMember' column to a DataFrame
def extract_member_info(df):
    """
    Function to extract member information from the 'AskingMember' column.
    
    Parameters:
    df (pd.DataFrame): The DataFrame containing PMQs.
    
    Returns:
    pd.DataFrame: A DataFrame containing member information.
    """
    members_df = pd.json_normalize(df['AskingMember'])
    return members_df

# extract member information
members_df = extract_member_info(pmqs_2025)
# display the first few rows of the members DataFrame
members_df.head()

,MnisId,PimsId,Name,ListAs,Constituency,Status,Party,PartyId,PartyColour,PhotoUrl
0,4776,6418,Munira Wilson,"Wilson, Munira",Twickenham,Active,Liberal Democrat,17,faa01a,https://members-api.parliament.uk/api/Members/...
1,4124,5617,Chi Onwurah,"Onwurah, Chi",Newcastle upon Tyne Central and West,Active,Labour,15,d50000,https://members-api.parliament.uk/api/Members/...
2,345,1455,Sir Edward Leigh,"Leigh, Sir Edward",Gainsborough,Active,Conservative,4,0063ba,https://members-api.parliament.uk/api/Members/...
3,415,3727,Fabian Hamilton,"Hamilton, Fabian",Leeds North East,Active,Labour,15,d50000,https://members-api.parliament.uk/api/Members/...
4,5358,6774,John Cooper,"Cooper, John",Dumfries and Galloway,Active,Conservative,4,0063ba,https://members-api.parliament.uk/api/Members/...


In [53]:
# because the 'AskingMember' column is a list of dictionaries, we can use the 'json_normalize' function to flatten it into a DataFrame 

# the extracted DataFrame is the same length as the original DataFrame, so we can concatenate it with the original DataFrame
pmqs_2025 = pd.concat([pmqs_2025, members_df], axis=1)
# display the first few rows of the updated DataFrame
pmqs_2025.head()

,Id,QuestionType,QuestionText,Status,Number,TabledWhen,RemovedFromToBeAskedWhen,DeclarableInterestDetail,HansardLink,UIN,...,MnisId,PimsId,Name,ListAs,Constituency,Status,Party,PartyId,PartyColour,PhotoUrl
0,359524,2,If he will make a statement on his departmenta...,5,1,2025-03-26T00:00:00,None,None,,903557,...,4776,6418,Munira Wilson,"Wilson, Munira",Twickenham,Active,Liberal Democrat,17,faa01a,https://members-api.parliament.uk/api/Members/...
1,359585,1,What diplomatic steps he is taking to increase...,7,1,2025-03-26T00:00:00,2025-03-27T07:28:42.14,None,,903532,...,4124,5617,Chi Onwurah,"Onwurah, Chi",Newcastle upon Tyne Central and West,Active,Labour,15,d50000,https://members-api.parliament.uk/api/Members/...
2,358715,2,If he will make a statement on his departmenta...,5,2,2025-03-26T00:00:00,None,None,,903558,...,345,1455,Sir Edward Leigh,"Leigh, Sir Edward",Gainsborough,Active,Conservative,4,0063ba,https://members-api.parliament.uk/api/Members/...
3,359538,1,What assessment he has made of the potential i...,5,2,2025-03-26T00:00:00,None,None,,903533,...,415,3727,Fabian Hamilton,"Hamilton, Fabian",Leeds North East,Active,Labour,15,d50000,https://members-api.parliament.uk/api/Members/...
4,358019,1,Which development programmes he plans to maint...,8,3,2025-03-26T00:00:00,2025-03-27T08:01:00.67,None,,903534,...,5358,6774,John Cooper,"Cooper, John",Dumfries and Galloway,Active,Conservative,4,0063ba,https://members-api.parliament.uk/api/Members/...


In [58]:
# at this point, we could save the DataFrame to a CSV file or perform further analysis
# let's save the DataFrame to a CSV file
# create a directory to save the file
import os
if not os.path.exists('data'):
    os.makedirs('data')
# save the DataFrame to a CSV file
pmqs_2025.to_csv('data/pmqs_2025.csv', index=False)
# display a message to indicate that the file has been saved
print("PMQs data for 2025 has been saved to 'data/pmqs_2025.csv'.")

PMQs data for 2025 has been saved to 'data/pmqs_2025.csv'.
